In [ ]:
!pip install openai

In [ ]:
import pandas as pd
df = pd.read_csv("bank_support_train.csv")
df.head(5)

,Support Query,Top Category,Sub Category
0,Can you explain the monthly maintenance fee on...,Fees and Charges,Understanding Fees
1,"I was charged a fee for an ATM withdrawal, why?",Fees and Charges,Understanding Fees
2,How do I dispute a transaction fee I believe i...,Fees and Charges,Dispute Charges
3,Are there any fees for using online banking?,Fees and Charges,Understanding Fees
4,What are the charges for a wire transfer?,Fees and Charges,Understanding Fees


In [ ]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"Top Category": "' + row['Top Category'] + '", "Sub Category": "' + row['Sub Category'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['Support Query']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data

dataset = pd.read_csv('/content/bank_support_train.csv')
converted_data = convert_to_gpt35_format(dataset)
converted_data[0]['messages']

[{'role': 'user',
  'content': 'Can you explain the monthly maintenance fee on my account?'},
 {'role': 'assistant',
  'content': '{"Top Category": "Fees and Charges", "Sub Category": "Understanding Fees"}'}]

In [ ]:
import json
json.loads(converted_data[0]['messages'][-1]['content'])

{'Top Category': 'Fees and Charges', 'Sub Category': 'Understanding Fees'}

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=dataset['Top Category'],
    random_state=42
)


In [ ]:
type(train_data[0])

dict

In [ ]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

In [ ]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

Training file id: file-II0txaz1Fn5AHk9pW2hcNF4H
Validation file id: file-QdtURL5p5IuEia7i6OsdyzrJ


In [ ]:
suffix_name = "fine-tuning-llm"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

FineTuningJob(id='ftjob-Zw1GQyKuidLp2l9qWxK0vekj', created_at=1709579364, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pbaCkW49bTYFJQz3RIDIduq8', result_files=[], status='validating_files', trained_tokens=None, training_file='file-II0txaz1Fn5AHk9pW2hcNF4H', validation_file='file-QdtURL5p5IuEia7i6OsdyzrJ', user_provided_suffix='fine-tuning-llm')

In [ ]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Zw1GQyKuidLp2l9qWxK0vekj', created_at=1709579364, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pbaCkW49bTYFJQz3RIDIduq8', result_files=[], status='validating_files', trained_tokens=None, training_file='file-II0txaz1Fn5AHk9pW2hcNF4H', validation_file='file-QdtURL5p5IuEia7i6OsdyzrJ', user_provided_suffix='fine-tuning-llm'), FineTuningJob(id='ftjob-Him25J9q2QSRUll9IxdJDkwY', created_at=1709495180, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:fine-tuning-llm:8ymDoqo2', finished_at=1709495735, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', orga

In [ ]:
response = client.fine_tuning.jobs.retrieve("ftjob-vEPbT0uj0tJtBSzVNip1qGPs")
response

FineTuningJob(id='ftjob-vEPbT0uj0tJtBSzVNip1qGPs', created_at=1709495072, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:fine-tuning-llm:8ymCqGFh', finished_at=1709495676, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pbaCkW49bTYFJQz3RIDIduq8', result_files=['file-elrHT9l6DaQFUhQPOSorWJoh'], status='succeeded', trained_tokens=9123, training_file='file-zz8dFQm4hcjTjo3QHlhAEGJa', validation_file='file-eLFj1yz0tFXXCF55nsJW1eM7', user_provided_suffix='fine-tuning-llm')

In [ ]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:fine-tuning-llm:8ymCqGFh


In [ ]:

def format_test(row):

    formatted_message = [
        {
            "role": "user",
            "content": row['Support Query']
        }
    ]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):

    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )

    return response.choices[0].message.content

In [ ]:
def store_predictions(test_df, fine_tuned_model_id):

    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")

In [ ]:
test_df = pd.read_csv("bank_support_train.csv")
test_df = test_df[1:10]
store_predictions(test_df, fine_tuned_model_id)

fine_tuned_model_id ft:gpt-3.5-turbo-0125:personal:fine-tuning-llm:8ymCqGFh
